In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image, ImageTk

# Load the dataset
books_data = pd.read_csv('C:\\Users\\Nadir\\Downloads\\books.csv')

# Preprocess the data
books_data['title'] = books_data['title'].fillna('')
books_data['authors'] = books_data['authors'].fillna('')
books_data['Genres'] = books_data['Genres'].fillna('')

# Combine relevant information into a single column
books_data['combined_features'] = books_data['title'] + ' ' + books_data['authors'] + ' ' + books_data['Genres']

# Use TfidfVectorizer to convert the combined features into numerical data
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(books_data['combined_features'])

# Compute the cosine similarity between books
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Initialize liked column with 0
books_data['liked'] = 0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, books_data['liked'], test_size=0.2, random_state=42)

# Train a k-Nearest Neighbors classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)


def predict_like_knn(book_title, author_name, genre):
    input_features = tfidf_vectorizer.transform([f'{book_title} {author_name} {genre}'])
    prediction = knn_classifier.predict(input_features)
    return prediction[0]


# Function to get book recommendations
def get_recommendations_knn(book_title, author_name, genre, cosine_sim_matrix, data):
    if author_name:
        # Filter data based on the specified author
        filtered_data = data[data['authors'].str.contains(author_name, case=False)]
        if filtered_data.empty:
            return pd.DataFrame()
    else:
        # If no author is specified, consider all books
        filtered_data = data

    if book_title:
        # If book title is specified, further filter based on title
        filtered_data = filtered_data[filtered_data['title'].str.contains(book_title, case=False)]

    if genre:
        # If genre is specified, further filter based on genre
        genres_list = genre.split(', ')
        for genre in genres_list:
            filtered_data = filtered_data[filtered_data['Genres'].str.contains(genre, case=False)]

    if filtered_data.empty:
        return pd.DataFrame()

    # Get the pairwise similarity scores of all books
    sim_scores = cosine_sim_matrix[filtered_data.index]

    # Sort the books based on the similarity scores
    rankings = pd.Series(sim_scores.sum(axis=1), name='rankings', index=filtered_data.index)
    rankings = rankings.sort_values(ascending=False)

    # Return the top 10 most similar books along with additional information
    recommendations = data.loc[rankings.index[:10], ['id', 'original_publication_year', 'average_rating', 'title', 'authors', 'Genres']]

    return recommendations


# GUI classes
class BookRecommendationGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Book Recommendation System")
        self.root.geometry("600x470")

        bg_image = Image.open("bg3.jpeg")
        bg_image = bg_image.resize((600, 470))
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        self.bg_label = ttk.Label(root, image=self.bg_photo)
        self.bg_label.place(relwidth=1, relheight=1)

       # self.root.configure(bg="#E6E6FA")

        self.title_label = ttk.Label(root, text="Book Recommendation System",
                                     font=('Helvetica', 24, 'bold'), background="#E6E6FA", foreground="#191970")
        self.title_label.pack(pady=20)

        self.book_label = ttk.Label(root, text="Enter Book Title:", font=('Helvetica', 14), background="#E6E6FA")
        self.book_entry = ttk.Entry(root, width=40, font=('Helvetica', 12))
        self.book_label.pack(pady=10)
        self.book_entry.pack(pady=10)

        self.author_label = ttk.Label(root, text="Enter Author Name:", font=('Helvetica', 14), background="#E6E6FA")
        self.author_entry = ttk.Entry(root, width=40, font=('Helvetica', 12))
        self.author_label.pack(pady=10)
        self.author_entry.pack(pady=10)

        self.genre_label = ttk.Label(root, text="Enter Genres (comma-separated):", font=('Helvetica', 14),
                                     background="#E6E6FA")
        self.genre_entry = ttk.Entry(root, width=40, font=('Helvetica', 12))
        self.genre_label.pack(pady=10)
        self.genre_entry.pack(pady=10)

        self.next_button = ttk.Button(root, text="Recommend Books", command=self.open_recommendation_gui,
                                      style="Accent.TButton")
        self.next_button.pack(pady=20)

        style = ttk.Style()
        style.configure("TButton", padding=10, font=('Helvetica', 14))
        style.configure("Accent.TButton", background="#8A2BE2", foreground="#0d0d0d", padding=10,
                        font=('Helvetica', 14, 'bold'))

    def open_recommendation_gui(self):
        book_title = self.book_entry.get()
        author_name = self.author_entry.get()
        genre = self.genre_entry.get()

        if book_title or author_name or genre:
            self.root.destroy()
            BookRecommendationResultGUI(book_title, author_name, genre)
        else:
            messagebox.showinfo("Error", "Please enter a book title, author name, or genre.")


class BookRecommendationResultGUI:
    def __init__(self, book_title, author_name, genre):
        self.book_title = book_title
        self.author_name = author_name
        self.genre = genre

        self.root = tk.Tk()
        self.root.title("Book Recommendations")
        self.root.geometry("900x600")
        self.root.configure(bg="#E6E6FA")

        self.result_label = ttk.Label(self.root, text="Recommendations:", font=('Helvetica', 24, 'bold'),
                                      background="#E6E6FA", foreground="#191970")
        self.result_label.pack(pady=20)

        columns = ("Book Title", "Original PublicationYear", "Average Rating", "Authors", "Genres")
        self.result_treeview = ttk.Treeview(self.root, columns=columns, show="headings", height=20)

        for col in columns:
            self.result_treeview.heading(col, text=col)
            self.result_treeview.column(col, width=150)

        self.show_recommendations()

        vertical_scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.result_treeview.yview)
        vertical_scrollbar.pack(side="right", fill="y")

        horizontal_scrollbar = ttk.Scrollbar(self.root, orient="horizontal", command=self.result_treeview.xview)
        horizontal_scrollbar.pack(side="bottom", fill="x")

        self.result_treeview.configure(yscrollcommand=vertical_scrollbar.set, xscrollcommand=horizontal_scrollbar.set)

        self.result_treeview.pack(pady=20, ipadx=10, ipady=10, fill=tk.BOTH, expand=True)

    def show_recommendations(self):
        recommendations = get_recommendations_knn(self.book_title, self.author_name, self.genre, cosine_sim, books_data)
        if not recommendations.empty:
            for _, row in recommendations.iterrows():
                self.result_treeview.insert("", "end", values=(
                    row["title"], row["original_publication_year"], row["average_rating"], row["authors"],
                    row["Genres"]))
        else:
            messagebox.showinfo("No Recommendations",
                                "No book recommendations found for the given title, author name, and genre.")


if __name__ == "__main__":
    root = tk.Tk()
    root.resizable(False, False)
    app = BookRecommendationGUI(root)
    root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'bg3.jpeg'